In [0]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [0]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 358kB 18.3MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=4ac4993701f79f5a73c8465a24af6478a1bb163d5f6f2c1da1500fd705a7697b
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for flask-compress: filename=Flask_Compress-1.5.0-cp36-none-any.whl size=5273 sha256=31202a19ee3fa8249f5e5327b0dd5e3e0ca10930b39355569ca4a901605ae050
  Stored in directory: /root/.cache/pip/wheels/f7/e9/e4/5afc286be7c87461375e33152558415dfeb0c8f5af3b50e742
Successfully built GPUtil flask-compress


In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2020-04-30 03:32:47--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.14.112, 2607:f8b0:4007:802::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.14.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  99.4MB/s    in 3.9s    

2020-04-30 03:32:51 (99.4 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [0]:
!nohup bert-serving-start -max_seq_len NONE -pooling_strategy NONE -num_worker=4 -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [0]:
from bert_serving.client import BertClient
bc = BertClient()

In [0]:
from google.colab import files
uploaded = files.upload()


In [0]:
import sys
np.set_printoptions(formatter={'float_kind':'{:f}'.format},threshold=sys.maxsize)

In [0]:
import os
import tensorflow
import numpy as np
from pathlib import Path
from bert_serving.client import BertClient
bc = BertClient()
os.chdir(r'/content/reddit')
f1=open('/content/BERT_Vocabulary.txt', 'r', encoding="utf8").read()
for fileName in Path('.').glob('*.txt'):
  f=open(str(fileName.absolute()),'r',encoding='utf-8').read().split(" ")
  a=str(fileName)
  f2=open("/content/out.txt","a+")
  for i in f:
    if "wiki_" in i.lower():
      x=i.rpartition("__")[0].replace("Wiki__","").replace("_"," ").lower().strip()
      if x=='':
        break
      for p in x:
        if p==" ":
          break
      if p!=" ":
        if x in f1:
          vec=bc.encode([x])
          c=str(vec.argmax())
          f2.write(a+','+x+','+c+'\n')
f2.close()

In [0]:
import re
import os
from pathlib import Path
os.chdir(r'/content/reddit')

def search(target, text, context=9):
    words = re.findall(r'\w+', text)
    matches = (i for (i,w) in enumerate(words) if w.lower() == target)
    for index in matches:
        if index < context //2:
            yield words[0:context+1]
        elif index > len(words) - context//2 - 1:
            yield words[-(context+1):]
        else:
            yield words[index - context//2:index + context//2 + 1]

def li(s):  
    str = ""    
    for i in s:
      for j in i:  
        str += j
      str+=' '
    return str

for fileName in Path('.').glob('*.txt'):
  a=str(fileName)
  text=open(a,'r',encoding='utf-8').read().lower()
  f=open(str(fileName.absolute()),'r',encoding='utf-8').read().split(" ")
  f2=open("/content/output.txt","a+")
  for i in f:
    if "wiki_" in i.lower():
      x=i.rpartition("__")[0].replace("Wiki__","").replace("_"," ").lower().strip()
      b=list(search(i.lower(),text))
      c=len(b)
      for i in range(c):
        f2.write(a+','+li(b[i])+','+x+'\n')
f2.close()

In [0]:
import hashlib
import os

os.chdir(r'/content')
completed_lines_hash = set()
f1=open("output.txt", "r")
f2=open("output_no_duplicates.txt", "w")

for line in f1:
  
  hashValue = hashlib.md5(line.rstrip().encode('utf-8')).hexdigest()
  
  if hashValue not in completed_lines_hash:
    f2.write(line)
    completed_lines_hash.add(hashValue)
f1.close()
f2.close()

In [0]:
f1=open('output_no_duplicates.txt','r')
f2=open('BERT_Vocabulary.txt','r').read()
f3=open('output_single_compared.txt','w')
for i in f1:
  a=i.split(',')[0].strip()
  b=i.split(',')[1].strip()
  c=i.split(',')[2].strip()
  if len(c.split())==1:
    if c in f2:
      f3.write(a+','+b+','+c+'\n')
f1.close()
f3.close()

In [0]:
f1=open('output_single_compared.txt','r')
f2=open('output_single_compared_sort.txt','w')
a=sorted(f1)
s=""
for i in a:
    s += i
f2.write(s)
f1.close()
f2.close()

In [0]:
f1=open('output_single_compared_sort.txt','r')
f2=open('output_clean.txt','w')

def li(s):  
    str = ""    
    for i in s:
      for j in i:  
        str += j
      str+=' '
    return str

for i in f1:
  z=0
  a=i.split(',')[0].strip()
  b=i.split(',')[1].strip()
  c=i.split(',')[2].strip()
  b=b.split()
  for j in b:
    if "wiki_" in j.lower():
      x=j.rpartition("__")[0].replace("wiki__","").replace("_"," ").lower().strip()
      b[z]=x
    z+=1
  f2.write(a+','+li(b)+','+c+'\n')
f1.close()
f2.close()